In [40]:
import pandas as pd
import os
from re import search

In [41]:
os.chdir("c:\\Users\\User\\Desktop\\Biochem") 

In [42]:
os.listdir()

['~$БХ_данные_pandas.xlsx', 'БХ_данные_pandas.xlsx']

In [55]:
# Загружаем данные из файла Excel 
df = pd.read_excel("c:\\Users\\User\\Desktop\\Biochem\\БХ_данные_pandas.xlsx", sheet_name="12.11.2023")
df1 = pd.read_excel("c:\\Users\\User\\Desktop\\Biochem\\БХ_данные_pandas.xlsx", sheet_name="19.11.2023")
df2 = pd.read_excel("c:\\Users\\User\\Desktop\\Biochem\\БХ_данные_pandas.xlsx", sheet_name="26.11.2023")
df3 = pd.read_excel("c:\\Users\\User\\Desktop\\Biochem\\БХ_данные_pandas.xlsx", sheet_name="02.12.2023")

In [56]:
# Списки концентраций
cons_list = ["3", "6", "12"]

In [58]:
# Добавляем столбец с датой к каждому датафрейму перед объединением
df['Дата'] = "12.11.2023"
df1['Дата'] = "19.11.2023"
df2['Дата'] = "26.11.2023"
df3['Дата'] = "02.12.2023"

# Теперь, когда у каждого датафрейма есть столбец 'Дата', можно объединять их
df_eritrociti_all = pd.concat([
    df[df['исслед.материал'] == 'эритроциты'],
    df1[df1['исслед.материал'] == 'эритроциты'],
    df2[df2['исслед.материал'] == 'эритроциты'],
    df3[df3['исслед.материал'] == 'эритроциты']
], ignore_index=True)

df_teni_all = pd.concat([
    df[df['исслед.материал'] == 'тени'],
    df1[df1['исслед.материал'] == 'тени'],
    df2[df2['исслед.материал'] == 'тени'],
    df3[df3['исслед.материал'] == 'тени']
], ignore_index=True)

df_vezykuli_all = pd.concat([
    df[df['исслед.материал'] == 'везикулы'],
    df1[df1['исслед.материал'] == 'везикулы'],
    df2[df2['исслед.материал'] == 'везикулы'],
    df3[df3['исслед.материал'] == 'везикулы']
], ignore_index=True)


In [59]:
print(df_eritrociti_all.head())

  исслед.материал   3мМ  опт.плотность_3   6мМ  опт.плотность_6   12мМ  \
0      эритроциты    К3            0.103    К6            0.108    К12   
1      эритроциты     1            0.262     1            0.263      1   
2      эритроциты     2            0.253     2            0.278      2   
3      эритроциты    Б3            0.213    Б6            0.241    Б12   
4      эритроциты  Б3,1            0.216  Б6,1            0.258  Б12,1   

   опт.плотность_12        Дата  
0             0.112  12.11.2023  
1             0.291  12.11.2023  
2             0.301  12.11.2023  
3             0.284  12.11.2023  
4             0.264  12.11.2023  


In [53]:
def calculate_values_erit(df, cons_list, sheet_date):
    results_erit = []
    for c in cons_list:
        H = [x for x in df.columns if search(f"_{c}", x)]
        if H:
            vez = df.set_index(H[0])
            for index, row in vez.iterrows():
                try:
                    OAA = (row[1] + row[2]) / 2 - row[f'К{c}']
                    MA = (row[f'Б{c}'] + row[f'Б{c},1']) / 2 - row[f'К{c}']
                    NKA = OAA - MA
                    MA_100 = MA / (OAA / 100)
                    summ = MA * 0.23 * 3 * 2 * 10 * 1.1 * 10
                    summ2 = NKA * 0.23 * 3 * 2 * 10 * 1.1 * 10
                    
                    results_erit.append({
                        'date': sheet_date,
                        'material_type': 'эритроциты',
                        'concentration': c + 'мМ',
                        'MA': summ,
                        'NKA': summ2
                    })
                except KeyError:
                    continue  # Пропускаем, если не находим нужные ключи
    return results_erit

def calculate_values_teni(df, cons_list, sheet_date):
    results_teni = []
    for c in cons_list:
        H = [x for x in df.columns if search(f"_{c}", x)]
        if H:
            vez = df.set_index(H[0])
            for index, row in vez.iterrows():
                try:
                    OAA = (row[1] + row[2]) / 2 - row[f'К{c}']
                    MA = (row[f'Б{c}'] + row[f'Б{c},1']) / 2 - row[f'К{c}']
                    NKA = OAA - MA
                    MA_100 = MA / (OAA / 100)
                    belok = 0.278967742
                    summ = MA * 0.23 * 3 * 2 / belok
                    summ2 = NKA * 0.23 * 3 * 2 / belok
                    
                    results_teni.append({
                        'date': sheet_date,
                        'material_type': 'тени',
                        'concentration': c + 'мМ',
                        'MA': summ,
                        'NKA': summ2
                    })
                except KeyError:
                    continue
    return results_teni

def calculate_values_vezykuli(df, cons_list, sheet_date):
    results_vezykuli = []
    for c in cons_list:
        H = [x for x in df.columns if search(f"_{c}", x)]
        if H:
            vez = df.set_index(H[0])
            for index, row in vez.iterrows():
                try:
                    OAA = (row[1] + row[2]) / 2 - row[f'К{c}']
                    MA = (row[f'Б{c}'] + row[f'Б{c},1']) / 2 - row[f'К{c}']
                    NKA = OAA - MA
                    MA_100 = MA / (OAA / 100)
                    belok = 0.278967742
                    summ = MA * 0.23 * 3 * 2 / belok
                    summ2 = NKA * 0.23 * 3 * 2 / belok
                    
                    results_vezykuli.append({
                        'date': sheet_date,
                        'material_type': 'везикулы',
                        'concentration': c + 'мМ',
                        'MA': summ,
                        'NKA': summ2
                    })
                except KeyError:
                    continue
    return results_vezykuli


In [60]:
res_eritrociti_all = calculate_values_erit(df_eritrociti_all, cons_list, '12.11.2023')
res_teni_all = calculate_values_teni(df_teni_all, cons_list, '12.11.2023')
res_vezykuli_all = calculate_values_vezykuli(df_vezykuli_all, cons_list, '12.11.2023')

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
# Выводим результаты
print("Результаты для эритроцитов:")
print(pd.DataFrame(res_erit_list))

In [ ]:
print("\nРезультаты для теней:")
print(pd.DataFrame(res_teni_list))

In [ ]:
print("\nРезультаты для везикул:")
print(pd.DataFrame(res_vezykuli_list))